In [12]:
# Important imports
import numpy as np
import pandas as pd
import kagglehub
import glob
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
import os
from dotenv import load_dotenv

In [2]:
#first dataset
from ucimlrepo import fetch_ucirepo
automobile = fetch_ucirepo(id=10)
df = automobile.data.features

In [3]:
#second dataset


# Download latest version
path = kagglehub.dataset_download("doaaalsenani/usa-cers-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\patri\.cache\kagglehub\datasets\doaaalsenani\usa-cers-dataset\versions\1


In [4]:
#third dataset

# Download latest version
path = kagglehub.dataset_download("nehalbirla/vehicle-dataset-from-cardekho")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\patri\.cache\kagglehub\datasets\nehalbirla\vehicle-dataset-from-cardekho\versions\4


In [5]:
print("📥 Loading UCI Automobile dataset...")
from ucimlrepo import fetch_ucirepo


# Fetch dataset
automobile = fetch_ucirepo(id=10)

# Get features and targets
df_features = automobile.data.features
df_targets = automobile.data.targets

# Combine them (if targets exist)
if df_targets is not None:
    df = pd.concat([df_features, df_targets], axis=1)
else:
    df = df_features

print(f"✅ Loaded {len(df)} rows from UCI")
print(f"Columns: {list(df.columns)}")

# Check if dataframe is actually populated
print(df.head())
print(df.info())

# Clean column names
df.columns = df.columns.str.lower().str.replace('-', '_').str.replace(' ', '_')

# Convert data types (important for PostgreSQL!)
# Handle numeric columns
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Replace NaN with None for SQL NULL
df = df.replace({np.nan: None})

# Upload to AWS RDS
load_dotenv()
password = os.getenv('DB_PASSWORD') or input("Enter password: ")
host = os.getenv('DB_HOST', 'car-analytics-db.copkksw0o3bx.us-east-1.rds.amazonaws.com')
user = os.getenv('DB_USER', 'ds5230_postgres')

password_encoded = quote_plus(password)
engine = create_engine(f'postgresql://{user}:{password_encoded}@{host}:5432/car_analytics_db')

# Upload (use 'replace' to overwrite the empty table)
df.to_sql('staging_uci_cars', engine, if_exists='replace', index=False)

print(f"✅ Uploaded {len(df)} rows to staging_uci_cars!")

📥 Loading UCI Automobile dataset...
✅ Loaded 205 rows from UCI
Columns: ['price', 'highway-mpg', 'city-mpg', 'peak-rpm', 'horsepower', 'compression-ratio', 'stroke', 'bore', 'fuel-system', 'engine-size', 'num-of-cylinders', 'engine-type', 'curb-weight', 'height', 'width', 'length', 'wheel-base', 'engine-location', 'drive-wheels', 'body-style', 'num-of-doors', 'aspiration', 'fuel-type', 'make', 'normalized-losses', 'symboling']
     price  highway-mpg  city-mpg  peak-rpm  horsepower  compression-ratio  \
0  13495.0           27        21    5000.0       111.0                9.0   
1  16500.0           27        21    5000.0       111.0                9.0   
2  16500.0           26        19    5000.0       154.0                9.0   
3  13950.0           30        24    5500.0       102.0               10.0   
4  17450.0           22        18    5500.0       115.0                8.0   

   stroke  bore fuel-system  engine-size  ...  wheel-base engine-location  \
0    2.68  3.47        

In [6]:
# Get database connection once
load_dotenv()
password = os.getenv('DB_PASSWORD') or input("Enter password: ")
host = os.getenv('DB_HOST', 'car-analytics-db.copkksw0o3bx.us-east-1.rds.amazonaws.com')
user = os.getenv('DB_USER', 'ds5230_postgres')
password = quote_plus(password)
engine = create_engine(f'postgresql://{user}:{password}@{host}:5432/car_analytics_db')

# Dataset 1: UCI Automobile (if you have it downloaded already)
# Skip if you don't have the CSV file

# Dataset 2: US Cars
print("\n📥 Dataset 2: US Cars")
path = kagglehub.dataset_download("doaaalsenani/usa-cers-dataset")
df_us = pd.read_csv(glob.glob(f"{path}/*.csv")[0])
df_us.columns = df_us.columns.str.lower().str.replace(' ', '_')
df_us.to_sql('staging_us_cars', engine, if_exists='replace', index=False)
print(f"✅ Loaded {len(df_us)} rows to staging_us_cars")

# Dataset 3: CarDekho
print("\n📥 Dataset 3: CarDekho")
path = kagglehub.dataset_download("nehalbirla/vehicle-dataset-from-cardekho")
df_cardekho = pd.read_csv(glob.glob(f"{path}/*.csv")[0])
df_cardekho.columns = df_cardekho.columns.str.lower().str.replace(' ', '_')
df_cardekho.to_sql('staging_cardekho', engine, if_exists='replace', index=False)
print(f"✅ Loaded {len(df_cardekho)} rows to staging_cardekho")

# Dataset 4: car-models
print("\n📥 Dataset 4: CarModels")
path = kagglehub.dataset_download("markmedhat/car-model")
df_cardekho = pd.read_csv(glob.glob(f"{path}/*.csv")[0])
df_cardekho.columns = df_cardekho.columns.str.lower().str.replace(' ', '_')
df_cardekho.to_sql('staging_carmodels', engine, if_exists='replace', index=False)
print(f"✅ Loaded {len(df_cardekho)} rows to staging_carmodels")

# Dataset 5: USA Mercedes Benz
print("\n📥 Dataset 5: USA Mercedes Benz")
path = kagglehub.dataset_download("danishammar/usa-mercedes-benz-prices-dataset")
df_cardekho = pd.read_csv(glob.glob(f"{path}/*.csv")[0])
df_cardekho.columns = df_cardekho.columns.str.lower().str.replace(' ', '_')
df_cardekho.to_sql('staging_us_mercedes', engine, if_exists='replace', index=False)
print(f"✅ Loaded {len(df_cardekho)} rows to staging_us_mercedes")


print("\n🎉 All datasets loaded!")


📥 Dataset 2: US Cars
✅ Loaded 2499 rows to staging_us_cars

📥 Dataset 3: CarDekho
✅ Loaded 301 rows to staging_cardekho

📥 Dataset 4: CarModels
✅ Loaded 13 rows to staging_carmodels

📥 Dataset 5: USA Mercedes Benz
✅ Loaded 2429 rows to staging_us_mercedes

🎉 All datasets loaded!


In [7]:
# Check what tables we have
tables_query = """
SELECT table_name, 
       (SELECT COUNT(*) FROM information_schema.columns WHERE table_name = t.table_name) as column_count
FROM information_schema.tables t
WHERE table_schema = 'public'
ORDER BY table_name;
"""


result = pd.read_sql(tables_query, engine)
print(result)

            table_name  column_count
0     staging_cardekho             9
1    staging_carmodels             4
2     staging_uci_cars            26
3      staging_us_cars            13
4  staging_us_mercedes             5


In [8]:
# Count rows in each table
counts = pd.read_sql("""
    SELECT 
        'staging_uci_cars' as table_name,
        COUNT(*) as row_count
    FROM staging_uci_cars
    UNION ALL
    SELECT 
        'staging_us_cars',
        COUNT(*)
    FROM staging_us_cars
    UNION ALL
    SELECT 
        'staging_cardekho',
        COUNT(*)
    FROM staging_cardekho;
""", engine)

print(counts)

         table_name  row_count
0  staging_uci_cars        205
1   staging_us_cars       2499
2  staging_cardekho        301


In [9]:
# Preview each dataset
print("=== UCI Cars ===")
print(pd.read_sql("SELECT * FROM staging_uci_cars LIMIT 3", engine))

print("\n=== US Cars ===")
print(pd.read_sql("SELECT * FROM staging_us_cars LIMIT 3", engine))

print("\n=== CarDekho ===")
print(pd.read_sql("SELECT * FROM staging_cardekho LIMIT 3", engine))

print("\n=== CarModels ===")
print(pd.read_sql("SELECT * FROM staging_carmodels LIMIT 3", engine))

print("\n=== USA Mercedes Benz ===")
print(pd.read_sql("SELECT * FROM staging_us_mercedes LIMIT 3", engine))

=== UCI Cars ===
     price  highway_mpg  city_mpg peak_rpm horsepower  compression_ratio  \
0  13495.0           27        21   5000.0      111.0                9.0   
1  16500.0           27        21   5000.0      111.0                9.0   
2  16500.0           26        19   5000.0      154.0                9.0   

  stroke  bore fuel_system  engine_size  ...  wheel_base engine_location  \
0   2.68  3.47        mpfi          130  ...        88.6           front   
1   2.68  3.47        mpfi          130  ...        88.6           front   
2   3.47  2.68        mpfi          152  ...        94.5           front   

   drive_wheels   body_style  num_of_doors  aspiration  fuel_type  \
0           rwd  convertible           2.0         std        gas   
1           rwd  convertible           2.0         std        gas   
2           rwd    hatchback           2.0         std        gas   

          make normalized_losses symboling  
0  alfa-romero              None         3  
1  alf

In [19]:
"""Processing and combine table staging_us_mercedes and staging_us_cars by executing PostgrestSQL queries"""
with engine.connect() as connection:
    # Create Dummy table     
    creation_query = text("""CREATE TABLE testing_table2 AS TABLE staging_us_mercedes; 
                            CREATE TABLE combined_table AS TABLE staging_us_cars;""")
    # Modifying dummy tables
    alteration_query = text("""ALTER TABLE testing_table2 ADD COLUMN IF NOT EXISTS year INTEGER;
        UPDATE testing_table2
        SET year = CAST(SUBSTRING(name FROM 1 FOR 4) AS INTEGER);
        ALTER TABLE testing_table2 ADD COLUMN brand text DEFAULT 'mercedes-benz';
        ALTER TABLE testing_table2 ADD COLUMN title_status text DEFAULT 'clean vehicle';
        delete from testing_table2 where price = 'Not Priced';
        ALTER TABLE testing_table2
        ALTER COLUMN price TYPE BIGINT
        USING (REPLACE(REPLACE(price, '$', ''), ',', '')::NUMERIC)::BIGINT;
        ALTER TABLE testing_table2
        ALTER COLUMN mileage TYPE BIGINT
        USING (REPLACE(REPLACE(mileage, ' mi', ''), ',', '')::NUMERIC)::BIGINT;
        ALTER TABLE testing_table2 DROP COLUMN rating;
        ALTER TABLE testing_table2 DROP COLUMN review_count;
        ALTER TABLE testing_table2 DROP COLUMN name;""")

    # inserting value from one table to another
    insertion_query = text("""INSERT INTO combined_table (mileage, price, year, brand, title_status)
        SELECT mileage, price, year, brand, title_status
        FROM testing_table2;""")
    connection.execute(creation_query)
    connection.execute(alteration_query)
    connection.execute(insertion_query)
    connection.commit()